In [1]:
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
import numpy as np

In [29]:
# Getting reviews with words that come under 5000, most occuring words in the entire corpus

vocab_size = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [30]:
# getting the index of all the words
word_idx = imdb.get_word_index()

# Originally the index number of a value and not a key,
# hence converting the index as key and the words as values
word_idx = {i: word for word, i in word_idx.items()}

# again printing the review
print([word_idx[i] for i in x_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [31]:
# Get the minimum and the maximum length of reviews
print("Max length of a review:: ", len(max((x_train+x_test), key=len)))
print("Min length of a review:: ", len(min((x_train+x_test), key=len)))

Max length of a review::  2697
Min length of a review::  70


Since now we know the minimum and maximum word count of a reivew, we need to pad the words with less number of reviews since a NN expects all the input in a fixed size.

In [32]:
from tensorflow.keras.preprocessing import sequence

# Keeping a fixed length of all reviews to max 400 words
max_words = 400

x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

x_valid, y_valid = x_train[:64], y_train[:64]
x_train_, y_train_ = x_train[64:], y_train[64:]

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Define parameters
embd_len = 32

# Create the RNN model
model = Sequential(name='Simple_RNN')
model.add(Embedding(vocab_size, embd_len, input_length=max_words))
model.add(SimpleRNN(128, activation='tanh', return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# build the model
model.build(input_shape=(None, max_words))  # Batch size is None, sequence length is max_words

# Print the model summary
print(model.summary())

Model: "Simple_RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ (None, 400, 32)             │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_4 (SimpleRNN)             │ (None, 128)                 │          20,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 180,737 (706.00 KB)

 Trainable params: 180,737 (706.00 KB)

 Non-trainable params: 0 (0.00 B)

None


In [34]:
# compile the model
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [35]:
# Train the model
history = model.fit(
    x_train_, y_train_,
    batch_size = 64,
    epochs = 5,
    verbose=1,
    validation_data = (x_valid, y_valid)
    )

Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 50s 124ms/step - accuracy: 0.5155 - loss: 0.6934 - val_accuracy: 0.5469 - val_loss: 0.6753
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 82s 123ms/step - accuracy: 0.7353 - loss: 0.5341 - val_accuracy: 0.6094 - val_loss: 0.6255
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 82s 123ms/step - accuracy: 0.7536 - loss: 0.5026 - val_accuracy: 0.7188 - val_loss: 0.5708
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 82s 123ms/step - accuracy: 0.7482 - loss: 0.5125 - val_accuracy: 0.7031 - val_loss: 0.5263
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 82s 124ms/step - accuracy: 0.7557 - loss: 0.5029 - val_accuracy: 0.7500 - val_loss: 0.4854


In [37]:
print("Simple RNN Score: ", model.evaluate(x_test, y_test, verbose=0))

Simple RNN Score:  [0.4681173861026764, 0.790120005607605]


In [38]:
# Defining GRU model
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocab_size,
						embd_len,
						input_length=max_words))
gru_model.add(GRU(128,
				activation='tanh',
				return_sequences=False))
gru_model.add(Dense(1, activation='sigmoid'))

# build the model
gru_model.build(input_shape=(None, max_words))  # Batch size is None, sequence length is max_words

# Printing the Summary
print(gru_model.summary())

# Compiling the model
gru_model.compile(
	loss="binary_crossentropy",
	optimizer='adam',
	metrics=['accuracy']
)

# Training the GRU model
history2 = gru_model.fit(x_train_, y_train_,
						batch_size=64,
						epochs=5,
						verbose=1,
						validation_data=(x_valid, y_valid))

# Printing model score on test data
print()
print("GRU model Score---> ", gru_model.evaluate(x_test, y_test, verbose=0))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "GRU_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ (None, 400, 32)             │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 128)                 │          62,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 222,337 (868.50 KB)

 Trainable params: 222,337 (868.50 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 174s 441ms/step - accuracy: 0.6499 - loss: 0.5884 - val_accuracy: 0.9062 - val_loss: 0.2726
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 201s 439ms/step - accuracy: 0.8796 - loss: 0.2957 - val_accuracy: 0.9062 - val_loss: 0.2743
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 201s 437ms/step - accuracy: 0.9061 - loss: 0.2391 - val_accuracy: 0.8750 - val_loss: 0.2471
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 202s 436ms/step - accuracy: 0.9294 - loss: 0.1897 - val_accuracy: 0.8906 - val_loss: 0.2407
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 170s 436ms/step - accuracy: 0.9481 - loss: 0.1444 - val_accuracy: 0.9062 - val_loss: 0.2128

GRU model Score--->  [0.3202770948410034, 0.8850399851799011]


In [39]:
# Defining Bidirectional LSTM model
bi_lstm_model = Sequential(name="Bidirectional_LSTM")
bi_lstm_model.add(Embedding(vocab_size,
							embd_len,
							input_length=max_words))
bi_lstm_model.add(Bidirectional(LSTM(128,
									activation='tanh',
									return_sequences=False)))
bi_lstm_model.add(Dense(1, activation='sigmoid'))

#built model
bi_lstm_model.build(input_shape=(None, max_words))

# Printing model summary
print(bi_lstm_model.summary())

# Compiling model summary
bi_lstm_model.compile(
loss="binary_crossentropy",
optimizer='adam',
metrics=['accuracy']
)

# Training the model
history3 = bi_lstm_model.fit(x_train_, y_train_,
							batch_size=64,
							epochs=5,
							verbose=2,
							validation_data=(x_test, y_test))

# Printing model score on test data
print()
print("Bidirectional LSTM model Score---> ",
	bi_lstm_model.evaluate(x_test, y_test, verbose=0))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "Bidirectional_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)             │ (None, 400, 32)             │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 256)                 │         164,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 325,121 (1.24 MB)

 Trainable params: 325,121 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
390/390 - 413s - 1s/step - accuracy: 0.7761 - loss: 0.4557 - val_accuracy: 0.8288 - val_loss: 0.4002
Epoch 2/5
390/390 - 495s - 1s/step - accuracy: 0.8788 - loss: 0.3032 - val_accuracy: 0.8725 - val_loss: 0.3101
Epoch 3/5
390/390 - 502s - 1s/step - accuracy: 0.8923 - loss: 0.2701 - val_accuracy: 0.8659 - val_loss: 0.3208
Epoch 4/5
390/390 - 505s - 1s/step - accuracy: 0.9151 - loss: 0.2215 - val_accuracy: 0.8687 - val_loss: 0.3306
Epoch 5/5
390/390 - 501s - 1s/step - accuracy: 0.9200 - loss: 0.2078 - val_accuracy: 0.8420 - val_loss: 0.4153

Bidirectional LSTM model Score--->  [0.41525277495384216, 0.8420400023460388]


**Conclusion**
* Almost all then models are performing ok but the best among them is Bidirectional LSTM
* This model also took a long time to finish it's training part.

Thank you for going through this till the end, feel free to get in touch if you have any questions... :)